In [ ]:
# ============================================================
# 🧰 STEP 0 — Install dependencies
# ============================================================
!pip -q install textattack regex arabic-reshaper python-bidi
!pip -q install --upgrade datasets accelerate

# ============================================================
# 📦 STEP 1 — Load dataset (auto-detect path)
#   Expected in file: dataset = [(text, label), ...]
# ============================================================
import importlib.util, os
from typing import Any

def import_dataset_module() -> Any:
    candidate_paths = [
        "/content/arbml_arabic_100_reviews.py"
    ]
    for p in candidate_paths:
        if os.path.exists(p):
            spec = importlib.util.spec_from_file_location("arbml_dataset", p)
            mod = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(mod)
            print(f"✅ Loaded dataset module from: {p}")
            return mod
    raise FileNotFoundError("❌ Could not find 'arbml_arabic_100_reviews.py'.")

arbml = import_dataset_module()
tt_pairs = list(arbml.dataset)  # [(text, label), ...]
print(f"Samples: {len(tt_pairs)} | Labels: {sorted(set(int(x[1]) for x in tt_pairs))}")

# Prepare HF dataset for fine-tuning
from datasets import Dataset as HFDataset
texts  = [str(x[0]) for x in tt_pairs]
labels = [int(x[1]) for x in tt_pairs]
hf_ds  = HFDataset.from_dict({"text": texts, "label": labels}).train_test_split(test_size=0.2, seed=42)
num_labels = len(set(labels))

# ============================================================
# 🤗 STEP 2 — Fine-tune AraBERT (version-safe) + disable W&B
# ============================================================
import numpy as np, torch, inspect

# 🔕 Disable Weights & Biases prompt + quiet tokenizers
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)

MODEL_NAME = "asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding=False, max_length=256)

tokenized = hf_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": (preds == labels_np).mean()}

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

# Build TrainingArguments kwargs that match installed transformers version
sig = inspect.signature(TrainingArguments.__init__)
argnames = set(sig.parameters.keys())

ta_kwargs = dict(
    output_dir="arabert-ft",
    num_train_epochs=3,               # bump to 5 for stronger model
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    seed=42,
    fp16=torch.cuda.is_available(),
)
if "evaluation_strategy" in argnames:
    ta_kwargs["evaluation_strategy"] = "epoch"
elif "eval_strategy" in argnames:
    ta_kwargs["eval_strategy"] = "epoch"
if "save_strategy" in argnames:
    ta_kwargs["save_strategy"] = "no"
elif "save_total_limit" in argnames:
    ta_kwargs["save_total_limit"] = 1

training_args = TrainingArguments(**ta_kwargs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("🚀 Fine-tuning AraBERT...")
trainer.train()
print("📊 Eval:", trainer.evaluate())

# ============================================================
# 🧪 STEP 3 — Wrap model for TextAttack
# ============================================================
from textattack.models.wrappers import HuggingFaceModelWrapper
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# ============================================================
# 🧩 STEP 4 — Low-skip Arabic DeepWordBug (diacritics + confusables + neighbor + ins/del×3)
# ============================================================
import random, regex as re
from textattack.transformations import CompositeTransformation, WordSwap
from textattack.attack_recipes import AttackRecipe
from textattack import Attack
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR

AR_LETTERS = list("ابتثجحخدذرزسشصضطظعغفقكلمنهوي")
AR_EXTRA   = list("أإآءؤئىةى")
AR_CHARS   = AR_LETTERS + AR_EXTRA

AR_CHAR_RE   = re.compile(r"[\p{Arabic}]")
DIACRITIC_RE = re.compile(r"[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]")

AR_NEIGHBORS = {
    "ا": ["أ","إ","آ","ى","ل","ر"], "ب": ["ن","ت","ي","ة"], "ت": ["ب","ث","ن","ا","ي"],
    "ث": ["ت","ج","ح"], "ج": ["ث","ح","خ"], "ح": ["ج","خ","ع"], "خ": ["ح","ع","غ"],
    "د": ["ذ","ر"], "ذ": ["د","ر","ز"], "ر": ["د","ذ","ز","ا"], "ز": ["ر","ذ","س"],
    "س": ["ز","ش","د","ص"], "ش": ["س","ص"], "ص": ["س","ض","ط"], "ض": ["ص","ط","ظ"],
    "ط": ["ص","ض","ظ","ق"], "ظ": ["ط","ض","ع"], "ع": ["غ","ظ","ح","خ"], "غ": ["ع","ف","ق"],
    "ف": ["غ","ق"], "ق": ["ف","ك","ط"], "ك": ["ق","ل","م"], "ل": ["ك","ا","م","ن"],
    "م": ["ن","ك","ل"], "ن": ["م","ب","ت"], "ه": ["ة","و","ي"], "و": ["ؤ","ه","ز"],
    "ي": ["ى","ئ","ب","ت","ن"], "ة": ["ه","ت"], "ى": ["ي","ا"], "ؤ": ["و","ء"], "ئ": ["ي","ء"],
}

def is_arabic_char(ch: str) -> bool:
    return bool(AR_CHAR_RE.match(ch))

class WordSwapArabicDiacriticToggle(WordSwap):
    COMMON_MARKS = ["َ","ُ","ِ","ّ","ْ"]
    def _get_replacement_words(self, word, **kwargs):
        if not any(is_arabic_char(c) for c in word): return []
        no_diac = DIACRITIC_RE.sub("", word)
        cands = set()
        if no_diac and no_diac != word: cands.add(no_diac)
        ar_positions = [i for i,c in enumerate(no_diac) if is_arabic_char(c)]
        if ar_positions:
            i = random.choice(ar_positions); d = random.choice(self.COMMON_MARKS)
            cands.add(no_diac[:i+1] + d + no_diac[i+1:])
        return list(cands)

CONFUSABLES = {
    "ا": ["أ","إ","آ","ى"], "أ": ["ا","إ","آ"], "إ": ["ا","أ","آ"], "آ": ["ا","أ","إ"],
    "ي": ["ى","ئ"], "ى": ["ي","ا"], "ئ": ["ي","ء"], "ه": ["ة"], "ة": ["ه","ت"], "و": ["ؤ"], "ؤ": ["و","ء"],
}
class WordSwapArabicConfusableSwap(WordSwap):
    def _get_replacement_words(self, word, **kwargs):
        cands = set()
        for i,ch in enumerate(word):
            if ch in CONFUSABLES:
                new = word[:i] + random.choice(CONFUSABLES[ch]) + word[i+1:]
                if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicNeighborSwap(WordSwap):
    def _get_replacement_words(self, word, **kwargs):
        cands = set()
        for i,ch in enumerate(word):
            if not is_arabic_char(ch): continue
            neigh = AR_NEIGHBORS.get(ch, [])
            if neigh:
                new = word[:i] + random.choice(neigh) + word[i+1:]
                if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicRandomInsertion(WordSwap):
    def __init__(self, max_insertions_per_word=3):
        super().__init__(); self.max_insertions_per_word = max_insertions_per_word
    def _get_replacement_words(self, word, **kwargs):
        if not any(is_arabic_char(c) for c in word): return []
        cands = set()
        for _ in range(self.max_insertions_per_word):
            pos = random.randint(0, len(word)); ins = random.choice(AR_CHARS)
            new = word[:pos] + ins + word[pos:]
            if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicRandomDeletion(WordSwap):
    def __init__(self, max_deletions_per_word=3):
        super().__init__(); self.max_deletions_per_word = max_deletions_per_word
    def _get_replacement_words(self, word, **kwargs):
        idx = [i for i,c in enumerate(word) if is_arabic_char(c)]
        if not idx: return []
        cands = set()
        for _ in range(self.max_deletions_per_word):
            i = random.choice(idx); new = word[:i] + word[i+1:]
            if new and new != word: cands.add(new)
        return list(cands)

from textattack.attack_recipes import AttackRecipe
class ArabicDeepWordBug(AttackRecipe):
    @staticmethod
    def build(model_wrapper):
        transformation = CompositeTransformation([
            WordSwapArabicDiacriticToggle(),
            WordSwapArabicConfusableSwap(),
            WordSwapArabicNeighborSwap(),
            WordSwapArabicRandomDeletion(max_deletions_per_word=3),
            WordSwapArabicRandomInsertion(max_insertions_per_word=3),
        ])
        constraints = [LevenshteinEditDistance(60)]  # larger budget → fewer "no edit" skips
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR()
        return Attack(goal_function, constraints, transformation, search_method)

attack = ArabicDeepWordBug.build(model_wrapper)
print("🧪 Built ArabicDeepWordBug.")

# ============================================================
# 🎯 STEP 5 — Attack ALL 100 samples from the attached dataset
# ============================================================
from textattack.datasets import Dataset as TADataset
from textattack import Attacker, AttackArgs
import random

TARGET_N = 100   # run on 100 items (or all if fewer)
all_pairs = list(zip(texts, labels))
random.Random(42).shuffle(all_pairs)
N = min(TARGET_N, len(all_pairs))

ta_test = TADataset(all_pairs[:N])

args = AttackArgs(
    num_examples=N,
    random_seed=42,
    disable_stdout=False,
)
attacker = Attacker(attack, ta_test, args)
print(f"⚔️  Running Arabic DeepWordBug on {N} examples (entire dataset)...")
attack_results = attacker.attack_dataset()
print("✅ Done.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape s

✅ Loaded dataset module from: /content/arbml_arabic_100_reviews.py
Samples: 100 | Labels: [0, 1]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

...

--------------------------------------------- Result 96 ---------------------------------------------
[[1 (54%)]] --> [[0 (50%)]]

OrderedDict({'text': 'فكرة الكتاب موفقة، كما أن [[المؤلف]] وفق في عرضها وصياغتها، فجزاه الله [[كل]] خير. [[نحن]] [[بحاجة]] لمثل هذه [[الأفكار]] الإبداعية [[التي]] تعرض لسيرة الرسول [[بطريقة]] خارجة [[عن]] المألوف وتساهم في تقديم نماذج من حياته صلى الله عليه وسلم للقراء بطريقة مختلفة'})

OrderedDict({'text': 'فكرة الكتاب موفقة، كما أن [[الملف]] وفق في عرضها وصياغتها، فجزاه الله [[كك]] خير. [[ثنحن]] [[بحاجت]] لمثل هذه [[عالأفكار]] الإبداعية [[للتي]] تعرض لسيرة الرسول [[بطريقه]] خارجة [[عةن]] المألوف وتساهم في تقديم نماذج من حياته صلى الله عليه وسلم للقراء بطريقة مختلفة'})


--------------------------------------------- Result 97 ---------------------------------------------
[[0 (51%)]] --> [[[SKIPPED]]]

OrderedDict({'text': 'هذه مراجعة ربما بدت فوضوية..لكن الرواية تركت أفكاري مشتتة ومبعثرة..فاعذروني. مافائدة الروابط إذا لم تتآلف الأرواح . الحياة ليست إلا مقا

[Succeeded / Failed / Skipped / Total] 66 / 7 / 25 / 98:  98%|█████████▊| 98/100 [01:52<00:02,  1.14s/it]

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (59%)]] --> [[0 (50%)]]

OrderedDict({'text': 'الأحداث هنا اكتر [[بكتير]] من الجزء الأول [[والأسلوب]] [[احلى]] [[كمان]] ،،عجبنى [[خياله]] [[جدا]] فى [[عالم]] [[الجن]] بصورة [[مختلفة]] تماما [[عن]] المعروف مستنيين العائد وشكلها [[هتتأخر]] وهننسى الأحداث'})

OrderedDict({'text': 'الأحداث هنا اكتر [[بغكتير]] من الجزء الأول [[والأسوب]] [[اذحلى]] [[كماآن]] ،،عجبنى [[خيالة]] [[ىجدا]] فى [[حالم]] [[ااجن]] بصورة [[مختلغة]] تماما [[ؤعن]] المعروف مستنيين العائد وشكلها [[ةتتأخر]] وهننسى الأحداث'})




[Succeeded / Failed / Skipped / Total] 67 / 7 / 26 / 100: 100%|██████████| 100/100 [01:53<00:00,  1.14s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (54%)]] --> [[1 (50%)]]

OrderedDict({'text': 'احب القصص الكلاسيكية ذات الطابع الريفي. اسلوب الكتابة الترجمة كان جيدا ولكن الوصف كان فاخرا لايناسب القصة واحسست ان التعريب اطال القصة [[اكثر]] مما ينبغي لحد [[الملل]] وهي قراءة لاتناسب فصل الصيف. :/. هناك بعض الكتب التي اسجل ملاحاظات واقتباسات جميلة منعا لتسربها من ذاكرتي ومع ماجدولين او تحت ظلال الزيزفون هناك وصف لادوراد لايهرب من فكري. وضل يقلب عينيه في السماء كأنه منجم يفتش عن النجم السابع والسبعين بعد المائة والخمسة والعشرين مليون. عن جد؟. النهاية كانت جميلة جدا وتاثرت بها [[نوعا]] ما. اذا كنت تعرف القليل عن قصة الحب مابين استيفن وماجدولين كل ماعليك قرائته هو صفحة الى النهاية. نجمتان فقط لو لم تكن مطولة [[اكثر]] من [[اللازم]] لاعطيتها [[ثلاث]] [[نجمات']]})

OrderedDict({'text': 'احب القصص الكلاسيكية ذات الطابع الريفي. اسلوب الكتابة الترجمة كان جيدا ولكن الوصف كان فاخرا لايناسب القصة واحسست ان التعريب اطال القصة [[اكر]] مما ينبغي ل

In [ ]:
# ============================================================
# 🧰 STEP 0 — Install dependencies
# ============================================================
!pip -q install textattack regex arabic-reshaper python-bidi
!pip -q install --upgrade datasets accelerate

# ============================================================
# 📦 STEP 1 — Load dataset (auto-detect path)
#   Expected in file: dataset = [(text, label), ...]
# ============================================================
import importlib.util, os
from typing import Any

def import_dataset_module() -> Any:
    candidate_paths = [
        "/content/arbml_arabic_100_reviews.py"
    ]
    for p in candidate_paths:
        if os.path.exists(p):
            spec = importlib.util.spec_from_file_location("arbml_dataset", p)
            mod = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(mod)
            print(f"✅ Loaded dataset module from: {p}")
            return mod
    raise FileNotFoundError("❌ Could not find 'arbml_arabic_100_reviews.py'.")

arbml = import_dataset_module()
tt_pairs = list(arbml.dataset)  # [(text, label), ...]
print(f"Samples: {len(tt_pairs)} | Labels: {sorted(set(int(x[1]) for x in tt_pairs))}")

# Prepare HF dataset for fine-tuning
from datasets import Dataset as HFDataset
texts  = [str(x[0]) for x in tt_pairs]
labels = [int(x[1]) for x in tt_pairs]
hf_ds  = HFDataset.from_dict({"text": texts, "label": labels}).train_test_split(test_size=0.2, seed=42)
num_labels = len(set(labels))

# ============================================================
# 🤗 STEP 2 — Fine-tune AraBERT (version-safe) + disable W&B
# ============================================================
import numpy as np, torch, inspect

# 🔕 Disable Weights & Biases prompt + quiet tokenizers
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)

MODEL_NAME = "asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding=False, max_length=256)

tokenized = hf_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": (preds == labels_np).mean()}

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

# Build TrainingArguments kwargs that match installed transformers version
sig = inspect.signature(TrainingArguments.__init__)
argnames = set(sig.parameters.keys())

ta_kwargs = dict(
    output_dir="arabert-ft",
    num_train_epochs=3,               # bump to 5 for stronger model
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    seed=42,
    fp16=torch.cuda.is_available(),
)
if "evaluation_strategy" in argnames:
    ta_kwargs["evaluation_strategy"] = "epoch"
elif "eval_strategy" in argnames:
    ta_kwargs["eval_strategy"] = "epoch"
if "save_strategy" in argnames:
    ta_kwargs["save_strategy"] = "no"
elif "save_total_limit" in argnames:
    ta_kwargs["save_total_limit"] = 1

training_args = TrainingArguments(**ta_kwargs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("🚀 Fine-tuning AraBERT...")
trainer.train()
print("📊 Eval:", trainer.evaluate())

# ============================================================
# 🧪 STEP 3 — Wrap model for TextAttack
# ============================================================
from textattack.models.wrappers import HuggingFaceModelWrapper
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# ============================================================
# 🧩 STEP 4 — Low-skip Arabic DeepWordBug (diacritics + confusables + neighbor + ins/del×3)
# ============================================================
import random, regex as re
from textattack.transformations import CompositeTransformation, WordSwap
from textattack.attack_recipes import AttackRecipe
from textattack import Attack
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR

AR_LETTERS = list("ابتثجحخدذرزسشصضطظعغفقكلمنهوي")
AR_EXTRA   = list("أإآءؤئىةى")
AR_CHARS   = AR_LETTERS + AR_EXTRA

AR_CHAR_RE   = re.compile(r"[\p{Arabic}]")
DIACRITIC_RE = re.compile(r"[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]")

AR_NEIGHBORS = {
    "ا": ["أ","إ","آ","ى","ل","ر"], "ب": ["ن","ت","ي","ة"], "ت": ["ب","ث","ن","ا","ي"],
    "ث": ["ت","ج","ح"], "ج": ["ث","ح","خ"], "ح": ["ج","خ","ع"], "خ": ["ح","ع","غ"],
    "د": ["ذ","ر"], "ذ": ["د","ر","ز"], "ر": ["د","ذ","ز","ا"], "ز": ["ر","ذ","س"],
    "س": ["ز","ش","د","ص"], "ش": ["س","ص"], "ص": ["س","ض","ط"], "ض": ["ص","ط","ظ"],
    "ط": ["ص","ض","ظ","ق"], "ظ": ["ط","ض","ع"], "ع": ["غ","ظ","ح","خ"], "غ": ["ع","ف","ق"],
    "ف": ["غ","ق"], "ق": ["ف","ك","ط"], "ك": ["ق","ل","م"], "ل": ["ك","ا","م","ن"],
    "م": ["ن","ك","ل"], "ن": ["م","ب","ت"], "ه": ["ة","و","ي"], "و": ["ؤ","ه","ز"],
    "ي": ["ى","ئ","ب","ت","ن"], "ة": ["ه","ت"], "ى": ["ي","ا"], "ؤ": ["و","ء"], "ئ": ["ي","ء"],
}

def is_arabic_char(ch: str) -> bool:
    return bool(AR_CHAR_RE.match(ch))

class WordSwapArabicDiacriticToggle(WordSwap):
    COMMON_MARKS = ["َ","ُ","ِ","ّ","ْ"]
    def _get_replacement_words(self, word, **kwargs):
        if not any(is_arabic_char(c) for c in word): return []
        no_diac = DIACRITIC_RE.sub("", word)
        cands = set()
        if no_diac and no_diac != word: cands.add(no_diac)
        ar_positions = [i for i,c in enumerate(no_diac) if is_arabic_char(c)]
        if ar_positions:
            i = random.choice(ar_positions); d = random.choice(self.COMMON_MARKS)
            cands.add(no_diac[:i+1] + d + no_diac[i+1:])
        return list(cands)

CONFUSABLES = {
    "ا": ["أ","إ","آ","ى"], "أ": ["ا","إ","آ"], "إ": ["ا","أ","آ"], "آ": ["ا","أ","إ"],
    "ي": ["ى","ئ"], "ى": ["ي","ا"], "ئ": ["ي","ء"], "ه": ["ة"], "ة": ["ه","ت"], "و": ["ؤ"], "ؤ": ["و","ء"],
}
class WordSwapArabicConfusableSwap(WordSwap):
    def _get_replacement_words(self, word, **kwargs):
        cands = set()
        for i,ch in enumerate(word):
            if ch in CONFUSABLES:
                new = word[:i] + random.choice(CONFUSABLES[ch]) + word[i+1:]
                if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicNeighborSwap(WordSwap):
    def _get_replacement_words(self, word, **kwargs):
        cands = set()
        for i,ch in enumerate(word):
            if not is_arabic_char(ch): continue
            neigh = AR_NEIGHBORS.get(ch, [])
            if neigh:
                new = word[:i] + random.choice(neigh) + word[i+1:]
                if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicRandomInsertion(WordSwap):
    def __init__(self, max_insertions_per_word=3):
        super().__init__(); self.max_insertions_per_word = max_insertions_per_word
    def _get_replacement_words(self, word, **kwargs):
        if not any(is_arabic_char(c) for c in word): return []
        cands = set()
        for _ in range(self.max_insertions_per_word):
            pos = random.randint(0, len(word)); ins = random.choice(AR_CHARS)
            new = word[:pos] + ins + word[pos:]
            if new != word: cands.add(new)
        return list(cands)

class WordSwapArabicRandomDeletion(WordSwap):
    def __init__(self, max_deletions_per_word=3):
        super().__init__(); self.max_deletions_per_word = max_deletions_per_word
    def _get_replacement_words(self, word, **kwargs):
        idx = [i for i,c in enumerate(word) if is_arabic_char(c)]
        if not idx: return []
        cands = set()
        for _ in range(self.max_deletions_per_word):
            i = random.choice(idx); new = word[:i] + word[i+1:]
            if new and new != word: cands.add(new)
        return list(cands)

from textattack.attack_recipes import AttackRecipe
class ArabicDeepWordBug(AttackRecipe):
    @staticmethod
    def build(model_wrapper):
        transformation = CompositeTransformation([
            WordSwapArabicDiacriticToggle(),
            WordSwapArabicConfusableSwap(),
            WordSwapArabicNeighborSwap(),
            WordSwapArabicRandomDeletion(max_deletions_per_word=3),
            WordSwapArabicRandomInsertion(max_insertions_per_word=3),
        ])
        constraints = [LevenshteinEditDistance(60)]  # larger budget → fewer "no edit" skips
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR()
        return Attack(goal_function, constraints, transformation, search_method)

attack = ArabicDeepWordBug.build(model_wrapper)
print("🧪 Built ArabicDeepWordBug.")

# ============================================================
# 🎯 STEP 5 — Attack ALL 100 samples from the attached dataset
# ============================================================
from textattack.datasets import Dataset as TADataset
from textattack import Attacker, AttackArgs
import random

TARGET_N = 100   # run on 100 items (or all if fewer)
all_pairs = list(zip(texts, labels))
random.Random(42).shuffle(all_pairs)
N = min(TARGET_N, len(all_pairs))

ta_test = TADataset(all_pairs[:N])

args = AttackArgs(
    num_examples=N,
    random_seed=42,
    disable_stdout=False,
)
attacker = Attacker(attack, ta_test, args)
print(f"⚔️  Running Arabic DeepWordBug on {N} examples (entire dataset)...")
attack_results = attacker.attack_dataset()
print("✅ Done.")


✅ Loaded dataset module from: /content/arbml_arabic_100_reviews.py
Samples: 100 | Labels: [0, 1]


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1741004209.py:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Fine-tuning AraBERT...


...

--------------------------------------------- Result 97 ---------------------------------------------
[[1 (51%)]] --> [[0 (50%)]]

OrderedDict({'text': 'هذه مراجعة ربما [[بدت]] فوضوية..لكن الرواية تركت أفكاري مشتتة ومبعثرة..فاعذروني. مافائدة الروابط إذا لم تتآلف الأرواح . الحياة ليست إلا مقامرة تلعب لمرة واحدة،وأنا خسرتها .ليس لي أن ألعب جولة أخرى . الناس لا يحتاجون للدعم المالي والمادي من بعضهم ، بل للحب والاهتمام. كل إنسان يكفيه إنسان واحد على كل حال..لكن عندما لا يكون هذا الإنسان موجودا ... في هذا المساء فهمت أن الإنسان قد يرتبط بإنسان آخر أكثر من ارتباطه بالحياة. هل يمكن للحياة أن تكون قاسية بهذه الطريقة؟. أن يعيش المرء حياته كلها و لا يحيا منها إلا خمسة أشهر قصيرة ؟ ... كيف يمكن لهذه القسوة أن تنعكس على روح الإنسان فتجعله يقسوعلى نفسه هكذا. ويتوقف عن الحياة ويمنع قلبه من العطاء والمحبة عقوبةلها على ذنب لم ترتكبه. أعجز عن التعبير عما شعرته تجاه هذه الرواية العظيمة..تحفة أدبية فلسفية استطاعت الغوص حتى العمق في النفس البشرية وتحليلها ورسم الشخصيات بواقعية كبيرة. اكتفيت ببعض الاقتباسا

[Succeeded / Failed / Skipped / Total] 67 / 14 / 17 / 98:  98%|█████████▊| 98/100 [04:36<00:05,  2.82s/it]

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (54%)]] --> [[0 (51%)]]

OrderedDict({'text': 'الأحداث [[هنا]] اكتر بكتير من الجزء الأول والأسلوب [[احلى]] كمان ،،عجبنى خياله جدا فى عالم [[الجن]] [[بصورة]] مختلفة تماما عن [[المعروف]] مستنيين العائد وشكلها [[هتتأخر]] [[وهننسى]] الأحداث'})

OrderedDict({'text': 'الأحداث [[صهنا]] اكتر بكتير من الجزء الأول والأسلوب [[انحلى]] كمان ،،عجبنى خياله جدا فى عالم [[ىلجن]] [[تصورة]] مختلفة تماما عن [[المعروغ]] مستنيين العائد وشكلها [[ةتتأخر]] [[وننسى]] الأحداث'})




[Succeeded / Failed / Skipped / Total] 68 / 14 / 18 / 100: 100%|██████████| 100/100 [04:39<00:00,  2.80s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (55%)]] --> [[1 (51%)]]

OrderedDict({'text': '[[احب]] القصص الكلاسيكية ذات الطابع الريفي. [[اسلوب]] الكتابة الترجمة كان جيدا ولكن الوصف كان فاخرا [[لايناسب]] [[القصة]] [[واحسست]] ان التعريب اطال القصة اكثر مما ينبغي لحد الملل [[وهي]] قراءة [[لاتناسب]] فصل الصيف. :/. هناك بعض الكتب التي اسجل ملاحاظات [[واقتباسات]] جميلة منعا [[لتسربها]] من ذاكرتي ومع [[ماجدولين]] او تحت ظلال الزيزفون هناك وصف [[لادوراد]] [[لايهرب]] من فكري. وضل يقلب عينيه في السماء كأنه منجم يفتش عن النجم السابع والسبعين بعد المائة والخمسة والعشرين مليون. عن جد؟. [[النهاية]] كانت جميلة جدا وتاثرت بها نوعا ما. اذا كنت تعرف القليل عن قصة الحب مابين استيفن [[وماجدولين]] كل ماعليك قرائته هو صفحة الى النهاية. نجمتان فقط لو لم تكن مطولة اكثر من اللازم لاعطيتها ثلاث نجمات'})

OrderedDict({'text': '[[اب]] القصص الكلاسيكية ذات الطابع الريفي. [[اسوب]] الكتابة الترجمة كان جيدا ولكن الوصف كان فاخرا [[ايناسب]] [[القدصة]] [[وحسس